In [ ]:
import sys
import os
import numpy as np
from numpy import array, asarray, ma, zeros, sum
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.cluster.vq import vq, kmeans, whiten, kmeans2
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

os.system('mkdir ./output')
os.system('mkdir ./output/Threats')
os.system('mkdir ./output/NoThreats')
os.system('mkdir ./training')
os.system('mkdir ./results')
os.system('mkdir ./models')
os.system('mkdir ./input')
print("#################################")
print("Directory application structure")
print("#################################")
print(check_output(["ls", "../input/datainput"]).decode("utf8"))
print(check_output(["ls", "../input/modelstraining"]).decode("utf8"))


# Any results you write to the current directory are saved as output.

In [ ]:
############################################################################################################
# Name: cluster_analisys
# Autor: Ramiro Bueno Martínez
# Date: 27/11/2017
##########################################################################################################
def cluster_analisys(subset):
	try:
		whitened = whiten(subset)
		codebook,distortion = kmeans(whitened,5)
		return distortion,whitened,codebook
	except Exception as exception:
		print ("cluster_analisys: Excepcion {0}".format(exception))

In [ ]:
PATH_PHOTO_FILES = '.'
NAME_FILE_MODEL_KMEANS = '../input/modelstraining/model_kmeans.csv'
DATA_FOR_PREDICTION_FILE = '../input/datainput/0a83698bce92a6824dcc37c1d7fc31f5.csv'

In [ ]:
############################################################################################################
# Name: creation_dataset_threats
# Autor: Ramiro Bueno Martínez
# Date: 27/11/2017
# Description: The goal of this function is the finding of possible anomalies in the different kind of
# images generated with the body scannes, using clustering techniques based in the analysis of different
# part of the body and different possition imagees
############################################################################################################
def creation_dataset_threats(dataset, FILENAME, highcontrast=False):

		# We have an image with a matrix structure of 660 rows by 512 colums
		# This function send this matrix to a algorithm to obtain different centroids and the distorsion
		# values that we will use to find possible anomalies 
		#Initialization variables
		m_distortion = 0
		col = 0
		row = 0
		nth = 0
		
		
		output_dir = PATH_PHOTO_FILES + '/' + 'output'
		input_dir = PATH_PHOTO_FILES + '/' + 'input'
		
		
		name_of_photo_file = input_dir + '/' + FILENAME
		
		
		print ("Starting clustering Analisys: {0}".format(FILENAME))
		
		#Only they are going to analyse 4 Image Front-Behind-Left-Right
		range_images = [0,4,8,12]
		try:
			
			#for nth in range(16):
			for nth in range_images:
			
				an_img = get_single_image(name_of_photo_file, nth)  				#returns the nth=3 image from the image stack
				
				#Prueba mejorando la resolución de la imagen 
				if highcontrast == True:
					img_rescaled = convert_to_grayscale(an_img)
					an_img = spread_spectrum(img_rescaled)
		
				data_array = np.array(an_img)
				distortion_array[nth,1],whitened,codebook= cluster_analisys(data_array)
				m_distortion = m_distortion + distortion_array[nth,1]
				record = pd.DataFrame([[FILENAME, nth, distortion_array[nth,1],'nothreat']],columns=['Name','Nth','Distortion','Threat'])
				dataset = dataset.append(record,ignore_index=True)
				col = col + 1
				if col % 4 == 0:
					row = row + 1
					col = 0
				print(".")
		
		
		
			#plt.show()
			#print(distortion_array)
			####################################################
			#Esta parte se puede optimizar con las funciones DataFrame.mean()
			media = 0
			media = m_distortion / 4
			#print ("Mean Value of Distortion: {0} \n".format(media))
			############################################################################################
			# At this point we are finding possible anomalies of different distortion measurement with
			# respect to a threshold fixed before
			############################################################################################
			percent = 17  # 17% of the mean value we suppose that its a wrong value 
			############################################################################################
			threshold_thread = media * percent / 100
			for i in range(len(dataset)):
				if abs(media - dataset.loc[i,('Distortion')]) > threshold_thread: 
					dataset.loc[i,('Threat')] = 'threat'
				if abs(media - dataset.loc[i,('Distortion')]) <= threshold_thread:
					dataset.loc[i,('Threat')] = 'nothreat'
					
		except Exception as exception:
			print ("Function [creation_dataset_threats]: Excepcion {0}".format(exception))
			sys.exit(1)
		finally:
			return dataset

In [ ]:
if __name__ == "__main__":

	print ("DATA_ANALISIS_TOOL: Initialization of the system...")
	try:
		file_models = os.listdir("../input/modelstraining")
        if 'model_kmeans.csv' in file_models:
			print('The training data has been created before .........')
		else:
			files = os.listdir('./training')
############################################################################################
#TRAINING MODEL SECTION
############################################################################################
		
		
		
			dataset_kmeans = pd.DataFrame(columns=['Id','Name','Nth','Distortion','Difference','Mean','Threat'])
			
			for name_file in files:
				dataset_kmeans = creation_dataset_threats(dataset_kmeans, name_file, highcontrast=True)
		
		
			distorsion_media = dataset_kmeans.mean(axis=None)['Distortion']
			for index in range(len(dataset_kmeans)):
				distortion = dataset_kmeans.iloc[index]['Distortion']
				dataset_kmeans.loc[index,('Difference')] = abs(distorsion_media - distortion)
				dataset_kmeans.loc[index,('Mean')] = distorsion_media
			
			save_model(dataset_kmeans,NAME_FILE_MODEL_KMEANS) 
			print("Created csv training data ....................")
		
		##############################################################################################
		#APPLICATION OF DIFFERENT KIND OF MODELS TO DETECT POSSIBLE THREATS
		##############################################################################################
		print("Starting model creation based in Support Vector Machine Algorithm ......")
		dataset = pd.read_csv(NAME_FILE_MODEL_KMEANS, sep=',')
		n_samp = len(dataset)
		X, y = make_classification(n_samples=n_samp, n_features=4, random_state=0)
        row = 0
		for index in range(len(dataset)):
			X[row][0]=dataset.loc[index,'Nth']
			X[row][1]=dataset.loc[index,'Distortion']
			X[row][2]=dataset.loc[index,'Difference']
			X[row][3]=dataset.loc[index,'Mean']
			if dataset.loc[index,'Threat'] == 'threat':
				y[index]=np.int32(1)
			else:
				y[index]=np.int32(0)
			
			row = row + 1
		clf = LinearSVC(random_state=0)
		clf.fit(X, y)
		print("Created model ..........")
		print("Starting the process of detection threats .....")
		
        DATA_FOR_PREDICTION_FILE
		dataset = pd.read_csv(DATA_FOR_PREDICTION_FILE, sep=',')
		data_array = np.array(dataset)
        distortion,whitened,codebook = cluster_analisys(data_array)
		#Generation of the record data
		mean_value = dataset['Distortion'].mean()
		difference = abs(mean_value - distortion)
		
        result_of_prediction = clf.predict([[nth,distortion,difference,mean_value]]) 	
		name_file = DATA_FOR_PREDICTION_FILE
        if result_of_prediction == True:
			print("Passenger {0} it could suppose a thread at Body Scanner {1}".format(name_file,3))
			threat_result = True
					
		else:
			print("Passenger {0} it could suppose a thread at Body Scanner {1}".format(name_file,3))
							
        
        sys.exit(0)  #At the demo only the programme do a unique prediction
  	
	except Exception as exception:
		print ("DATA_ANALISIS_TOOL: Excepcion {0}".format(exception))
		sys.exit(1)
	finally:
		print("DATA_ANALISIS_TOOL: That's all")